In [1]:
import datajoint as dj
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'twiceblackpink9.'
from RDBMS_tables import *

Connecting root@localhost:3306


Populating with mice

In [2]:
data = [
    {'mouse_id': 'BLA-Insc-1'},
    {'mouse_id': 'BLA-Insc-2'},
    {'mouse_id': 'BLA-Insc-3'},
    {'mouse_id': 'BLA-Insc-5'},
    {'mouse_id': 'BLA-Insc-6'},
    {'mouse_id': 'BLA-Insc-7'},
    {'mouse_id': 'BLA-Insc-8'},
    {'mouse_id': 'BLA-Insc-9'},
    {'mouse_id': 'BLA-Insc-11'},
    {'mouse_id': 'BLA-Insc-13'},
    {'mouse_id': 'BLA-Insc-14'},
    {'mouse_id': 'BLA-Insc-15'},
    {'mouse_id': 'BLA-Insc-16'},
    {'mouse_id': 'BLA-Insc-18'},
    {'mouse_id': 'BLA-Insc-19'}
]
mouse.insert(data)

DuplicateError: ("Duplicate entry 'BLA-Insc-1' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [3]:
mouse

mouse_id unique mouse_id
BLA-Insc-1
BLA-Insc-11
BLA-Insc-13
BLA-Insc-14
BLA-Insc-15
BLA-Insc-16
BLA-Insc-18
BLA-Insc-19
BLA-Insc-2
BLA-Insc-3


In [4]:
query = mouse.fetch()
query

array([('BLA-Insc-1',), ('BLA-Insc-11',), ('BLA-Insc-13',),
       ('BLA-Insc-14',), ('BLA-Insc-15',), ('BLA-Insc-16',),
       ('BLA-Insc-18',), ('BLA-Insc-19',), ('BLA-Insc-2',),
       ('BLA-Insc-3',), ('BLA-Insc-5',), ('BLA-Insc-6',), ('BLA-Insc-7',),
       ('BLA-Insc-8',), ('BLA-Insc-9',)], dtype=[('mouse_id', 'O')])